# App for Dataviz

In [10]:

import dash
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd


## import data

In [12]:
# read dataframe (preprocessed in main.ipynb)
df = pd.read_csv("data/merged_data.csv")
df.head()

,Unnamed: 0,average_life_expectancy,population,year,Country,Alpha-2 code,Alpha-3 code,id,Latitude (average),Longitude (average),Year,hiv_deaths_per_100k
0,0,50.331,12412000,1990,Afghanistan,AF,AFG,4,33.0,65.0,1990,0.366079
1,1,50.999,13299000,1991,Afghanistan,AF,AFG,4,33.0,65.0,1991,0.407195
2,2,51.641,14486000,1992,Afghanistan,AF,AFG,4,33.0,65.0,1992,0.435063
3,3,52.256,15817000,1993,Afghanistan,AF,AFG,4,33.0,65.0,1993,0.464077
4,4,52.842,17076000,1994,Afghanistan,AF,AFG,4,33.0,65.0,1994,0.498594


## Modify Data

In [13]:
# codes needed to shrink down the dataset to the african countries we want to display
african_iso_codes = ["DZA"
,"AGO"
,"BEN"
,"BWA"
,"BFA"
,"BDI"
,"CMR"
,"CPV"
,"CAF"
,"TCD"
,"COM"
,"COG"
,"COD"
,"CIV"
,"DJI"
,"EGY"
,"GNQ"
,"ERI"
,"ETH"
,"GAB"
,"GMB"
,"GHA"
,"GIN"
,"GNB"
,"KEN"
,"LSO"
,"LBR"
,"LBY"
,"MDG"
,"MLI"
,"MWI"
,"MRT"
,"MUS"
,"MYT"
,"MAR"
,"MOZ"
,"NAM"
,"NER"
,"NGA"
,"REU"
,"RWA"
,"STP"
,"SEN"
,"SYC"
,"SLE"
,"SOM"
,"ZAF"
,"SSD"
,"SDN"
,"SWZ"
,"TZA"
,"TGO"
,"TUN"
,"UGA"
,"ESH"
,"ZMB"
,"ZWE"]
# source: https://www.nro.net/list-of-country-codes-in-the-afrinic-region/

In [14]:
# shrink dataset to african countries
df = df[df["Alpha-3 code"].isin(african_iso_codes)]

In [15]:
df.head()

,Unnamed: 0,average_life_expectancy,population,year,Country,Alpha-2 code,Alpha-3 code,id,Latitude (average),Longitude (average),Year,hiv_deaths_per_100k
60,60,66.938,25759000,1990,Algeria,DZ,DZA,12,28.0,3.0,1990,0.205573
61,61,67.270,26400000,1991,Algeria,DZ,DZA,12,28.0,3.0,1991,0.227273
62,62,67.575,27028000,1992,Algeria,DZ,DZA,12,28.0,3.0,1992,0.250077
63,63,67.877,27636000,1993,Algeria,DZ,DZA,12,28.0,3.0,1993,0.274849
64,64,68.194,28214000,1994,Algeria,DZ,DZA,12,28.0,3.0,1994,0.300538


## Load Geojson data

In [24]:
# source: https://public.opendatasoft.com/explore/dataset/country_shapes/export/ 
import geojson
with open("data/country_shapes.geojson") as f:
    gj = geojson.load(f)
gj.get

<function FeatureCollection.get(key, default=None, /)>

## Create App

In [42]:
# the variable that holds our final webAPP
app = JupyterDash(__name__)

# create our map with animations already built in
chloropleth = px.choropleth(
        df, geojson=gj, color="hiv_deaths_per_100k",
        locations="Alpha-3 code", featureidkey="properties.cou_iso3_code",
        projection="mercator",animation_frame="year", 
        animation_group="Country", range_color=[df['hiv_deaths_per_100k'].min(), df["hiv_deaths_per_100k"].max()])
chloropleth.update_geos(fitbounds="locations", visible=False)
chloropleth.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, autosize = True)

# here we define the layout and components of our app
app.layout = html.Div([
    #  africa map container
    html.Div([
        html.P("HIV deaths per 100 Thousand people:"), # heading
        dcc.Graph(id="choropleth", figure=chloropleth, style={'width': '95vw', 'height': '90vh'}), # africa map figure with id "chloropleth"
    ], style={'width': '100%', 'height':'100%', 'margin': '0px 20px 20px 20px'}),
    # dropdown for country selection
    html.Div(dcc.Dropdown(
                    id='country_select',
                    options=[{'label': i, 'value': i} for i in african_iso_codes],
                    multi=True
                )),
    # graph (later for the wormgraph)
    html.Div([dcc.Graph(id="worm_figure")], style={'display':'block','margin': '20px 20px 20px 20px'}, id="worm_container")
            ])

## Create Callback

In [43]:
# here we define the functionality and interactivity of our App
# define input and output of callback function
@app.callback(
    # Output("choropleth", "figure"),
    Output("worm_figure", "figure"),
    # Output("worm_container", "style"),
    Input("country_select", "value"))
# the callback function: called when a value of the defined inputs above changes.
def display_choropleth(country_select):
    # worm_style = {'display', 'block'}
    # if any countries are selected
    if(country_select != None):
        fig2 = px.line(df[df["Alpha-3 code"].isin(country_select)], x="hiv_deaths_per_100k", y="average_life_expectancy")
        fig2.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, autosize = True)
        worm_style = {'display', 'block'}
    # else (should actually hide the bottom graph, but that did not work yet)
    else:
        fig2 = px.line(df,  x="year", y="average_life_expectancy")
    # return (what we have defined above)
    return fig2 #, fig2 , worm_style

## RUN APP Server

In [44]:
# run the local server
# visit http://127.0.0.1:8050/ in webbrowser to see results and error codes
app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
